In [1]:
import pandas as pd
from datasets import load_from_disk

/home/hawau.toyin/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
label = '/l/users/speech_lab/_SpeechT5PretrainDataset/Finetune/ASR/_labels/CLARTTS/dev.txt'
manifest = '/l/users/speech_lab/_SpeechT5PretrainDataset/Finetune/ASR/_manifest/CLARTTS/dev.tsv'

In [3]:
label = pd.read_csv(label, header=None, sep = '\t')
manifest = pd.read_csv(manifest,skiprows=1,sep='\t', header=None)

In [4]:
label.head()

,0
0,لأن فراق المألوف في العادة ومجانبة ما صار متفق...
1,ومن لطيف حكمته أن جعل لكل عبادة حالتين
2,من لم يشكر الإنعام فاعده من الأنعام
3,فيمقتوك ولا بكلام من هو دونك فيزدروك
4,وأما من بخل واستغنى وكذب بالحسنى


In [5]:
manifest.head()

,0,1
0,ch_14_arabic_tts_dataset_98.wav,88066
1,ch_05_arabic_tts_dataset_93.wav,64706
2,ch_12_arabic_tts_dataset_63.wav,57131
3,ch_16_arabic_tts_dataset_494.wav,61771
4,ch_10_arabic_tts_dataset_589.wav,56068


In [11]:
data = load_from_disk('/l/users/speech_lab/ClArTTS[ClArD_dataset]/ClArTTSBuckWalter/ClArD_HF_BuckWalter')['validation'].to_pandas()
data.head()

,audio_path,arabic_text,buckwalter_text,audio
0,<REPLACE_PATH>/ch_11_arabic_tts_dataset_328.wav,صَائِرٌ خَبَرًا فَكُنْ خَبَرًا يَرُوقُ جَمِيلَا,SaA}irN xabarFA fakuno xabarFA yaruwqu jamiylaA,"[-0.001159668, -0.0018615723, -0.001739502, -0..."
1,<REPLACE_PATH>/ch_09_arabic_tts_dataset_467.wav,أَذُكِرَتْ أَنْجَبَتْ الْتِمَاسُ مَنْ يَتَوَلَ...,>a*ukirato >anojabato AlotimaAsu mano yatawala...,"[-0.00039672852, -0.00061035156, -0.0004272461..."
2,<REPLACE_PATH>/ch_02_arabic_tts_dataset_192.wav,فَإِنَّ الْمَرْءَ عَدُوُّ مَا جَهِلَ وَأَنَا أ...,fa<ina~ Alomaro'a Eaduwu~ maA jahila wa>anaA >...,"[-0.00012207031, 0.000579834, 0.0005493164, -0..."
3,<REPLACE_PATH>/ch_18_arabic_tts_dataset_531.wav,وَالثَّانِي مَا تَجَاوَزَهُ إلَى غَيْرِهِ وَذَ...,waAl^a~Aniy maA tajaAwazahu <laY gayorihi wa*a...,"[0.0011901855, 0.0024719238, 0.0023498535, 0.0..."
4,<REPLACE_PATH>/ch_14_arabic_tts_dataset_330.wav,بِالْمَحْصُولِ مِنْ رِيَبِك وَهَذَا أَمْرٌ يَن...,biAlomaHoSuwli mino riyabik waha*aA >amorN yan...,"[-6.1035156e-05, -0.00021362305, -0.0001525878..."


In [12]:
import sys
import re
import sys
import unicodedata

import pandas as pd
import pyarabic.araby as araby
import sentencepiece as spm
from datasets import load_from_disk
import preprocessor as preprocess_tweet


import re
import pandas as pd

map_numbers = {'0': '٠', '1': '١', '2': '٢', '3': '٣', '4': '٤', '5': '٥', '6': '٦', '7': '٧', '8': '٨', '9': '٩'}
map_numbers = dict((v, k) for k, v in map_numbers.items())
punctuations = ''.join([chr(i) for i in list(i for i in range(sys.maxunicode) if unicodedata.category(chr(i)).startswith('P'))])
punctuations = punctuations + '÷#ݣ+=|$×⁄<>`åûݘ ڢ̇ پ'
# CORRECT BUCKWALTER
buckwalter = { #mapping from Arabic script to Buckwalter
    u'\u0628': u'b' , u'\u0630': u'*' , u'\u0637': u'T' , u'\u0645': u'm',
    u'\u062a': u't' , u'\u0631': u'r' , u'\u0638': u'Z' , u'\u0646': u'n',
    u'\u062b': u'^' , u'\u0632': u'z' , u'\u0639': u'E' , u'\u0647': u'h',
    u'\u062c': u'j' , u'\u0633': u's' , u'\u063a': u'g' , u'\u062d': u'H',
    u'\u0642': u'q' , u'\u0641': u'f' , u'\u062e': u'x' , u'\u0635': u'S',
    u'\u0634': u'$' , u'\u062f': u'd' , u'\u0636': u'D' , u'\u0643': u'k',
    u'\u0623': u'>' , u'\u0621': u'\'', u'\u0626': u'}' , u'\u0624': u'&',
    u'\u0625': u'<' , u'\u0622': u'|' , u'\u0627': u'A' , u'\u0649': u'Y',
    u'\u0629': u'p' , u'\u064a': u'y' , u'\u0644': u'l' , u'\u0648': u'w',
    u'\u064b': u'F' , u'\u064c': u'N' , u'\u064d': u'K' , u'\u064e': u'a',
    u'\u064f': u'u' , u'\u0650': u'i' , u'\u0651': u'~' , u'\u0652': u'o',
    u'ث': u'v', u'ث':u'V'
    }

ArabicScript = { #mapping from Buckwalter to Arabic script
    u'b': u'\u0628' , u'*': u'\u0630' , u'T': u'\u0637' , u'm': u'\u0645',
    u't': u'\u062a' , u'r': u'\u0631' , u'Z': u'\u0638' , u'n': u'\u0646',
    u'^': u'\u062b' , u'z': u'\u0632' , u'E': u'\u0639' , u'h': u'\u0647',
    u'j': u'\u062c' , u's': u'\u0633' , u'g': u'\u063a' , u'H': u'\u062d',
    u'q': u'\u0642' , u'f': u'\u0641' , u'x': u'\u062e' , u'S': u'\u0635',
    u'$': u'\u0634' , u'd': u'\u062f' , u'D': u'\u0636' , u'k': u'\u0643',
    u'>': u'\u0623' , u'\'': u'\u0621', u'}': u'\u0626' , u'&': u'\u0624',
    u'<': u'\u0625' , u'|': u'\u0622' , u'A': u'\u0627' , u'Y': u'\u0649',
    u'p': u'\u0629' , u'y': u'\u064a' , u'l': u'\u0644' , u'w': u'\u0648',
    u'F': u'\u064b' , u'N': u'\u064c' , u'K': u'\u064d' , u'a': u'\u064e',
    u'u': u'\u064f' , u'i': u'\u0650' , u'~': u'\u0651' , u'o': u'\u0652',
    u'v': u'ث', u'V':u'ث'
    }

def arabicToBuckwalter(word): #Convert input string to Buckwalter
    res = u''
    for letter in word:
        if(letter in buckwalter):
            res += buckwalter[letter]
        else:
            res += letter
    return res

def buckwalterToArabic(word): #Convert input string to Arabic
    res = u''
    for letter in word:
        if(letter in ArabicScript):
            res += ArabicScript[letter]
        else:
            res += letter
    return res

def convert_numerals_to_digit(word):
    sentence=[]
    for w in word:
        sentence.append(map_numbers.get(w, w))
    word = ''.join(sentence)
    return word

def remove_diacritics(word):
    return araby.strip_diacritics(word)
     

def remove_punctuation(word):
    return word.translate(str.maketrans('', '', re.sub('[@% ]','', punctuations))).lower()


def preprocess_arabic_text_v1(text):
    # text = preprocess_arabic_twitter_text(text)
    text = remove_diacritics(text)
    text = convert_numerals_to_digit(text)
    text = remove_punctuation(text)
    return text

In [13]:
data['audio_path'] = data['audio_path'].apply(lambda x: x.replace('<REPLACE_PATH>/', ''))
data['arabic_text'] = data['arabic_text'].apply(lambda x: preprocess_arabic_text_v1(x))

In [21]:
df = data[['audio_path', 'arabic_text']]
df.rename(columns= {'audio_path' : 0}, inplace=True)
df.head()

/tmp/ipykernel_2714737/2942947978.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns= {'audio_path' : 0}, inplace=True)


,0,arabic_text
0,ch_11_arabic_tts_dataset_328.wav,صائر خبرا فكن خبرا يروق جميلا
1,ch_09_arabic_tts_dataset_467.wav,أذكرت أنجبت التماس من يتولى تدبير المنزل
2,ch_02_arabic_tts_dataset_192.wav,فإن المرء عدو ما جهل وأنا أكره أن تكون عدو شيء...
3,ch_18_arabic_tts_dataset_531.wav,والثاني ما تجاوزه إلى غيره وذلك أربعة أشياء
4,ch_14_arabic_tts_dataset_330.wav,بالمحصول من ريبك وهذا أمر ينبغي للعاقل أن يضبط...


In [23]:
m_df = pd.merge(manifest, df, on = [0])

In [24]:
m_df.head()

,0,1,arabic_text
0,ch_14_arabic_tts_dataset_98.wav,88066,لأن فراق المألوف في العادة ومجانبة ما صار متفق...
1,ch_05_arabic_tts_dataset_93.wav,64706,ومن لطيف حكمته أن جعل لكل عبادة حالتين
2,ch_12_arabic_tts_dataset_63.wav,57131,من لم يشكر الإنعام فاعده من الأنعام
3,ch_16_arabic_tts_dataset_494.wav,61771,فيمقتوك ولا بكلام من هو دونك فيزدروك
4,ch_10_arabic_tts_dataset_589.wav,56068,وأما من بخل واستغنى وكذب بالحسنى


In [25]:
m_df[2] = 'speaker_embedding/CLARTTS_speaker_embedding.npy'
m_df[0] = m_df[0].apply(lambda x: 'test/' + x)
m_df.head()

,0,1,arabic_text,2
0,test/ch_14_arabic_tts_dataset_98.wav,88066,لأن فراق المألوف في العادة ومجانبة ما صار متفق...,speaker_embedding/CLARTTS_speaker_embedding.npy
1,test/ch_05_arabic_tts_dataset_93.wav,64706,ومن لطيف حكمته أن جعل لكل عبادة حالتين,speaker_embedding/CLARTTS_speaker_embedding.npy
2,test/ch_12_arabic_tts_dataset_63.wav,57131,من لم يشكر الإنعام فاعده من الأنعام,speaker_embedding/CLARTTS_speaker_embedding.npy
3,test/ch_16_arabic_tts_dataset_494.wav,61771,فيمقتوك ولا بكلام من هو دونك فيزدروك,speaker_embedding/CLARTTS_speaker_embedding.npy
4,test/ch_10_arabic_tts_dataset_589.wav,56068,وأما من بخل واستغنى وكذب بالحسنى,speaker_embedding/CLARTTS_speaker_embedding.npy


In [27]:
m_df[[0,1,2]].to_csv('/l/users/speech_lab/_SpeechT5PretrainDataset/Finetune/TTS/hubert_labels/CLARTTS/valid.tsv',sep='\t', header=None, index=False)

In [28]:
m_df['arabic_text'].to_csv('/l/users/speech_lab/_SpeechT5PretrainDataset/Finetune/TTS/labels/CLARTTS/valid.txt', header=None, index=False)

# ASC